In [ ]:
import numpy as np  
import matplotlib.pyplot as plt
import scipy.interpolate as spi

from utils import *



In [ ]:
### Initial data

rE = 6371 # Earth radius [km]
mu = 398600 # Earth gravitational parameter [km^3/s^2]

# Orbital parameters

h = 1000

a = rE + h # circular orbit
raan = np.deg2rad(50) # right ascension of the ascending node
i = np.deg2rad(55) # inclination
truan0 = 0 # true anomaly at t=0





Jc = np.array([[1800, 100, -200], [100, 2200, 300], [-200, 300, 2500]]) # Inertia matrix of the spacecraft [kg m^2]



# Inititial conditions for the attitude of the satellite
q0_1 = -0.1
q0_2 = -0.3
q0_3 = 0.5
q0_0 = np.sqrt(1 - ( q0_1**2 + q0_2**2 + q0_3**2))

q0 = Quat(q0_0, q0_1, q0_2, q0_3)

om0 = np.array([-0.01], [0.01], [0]) # Initial angular velocity [rad/s]

## Wheel parameters
Is = 1 # kg m^2
It = 0.5 # kg m^2

a1= np.array([1, 1, 1])/np.sqrt(3)
a2 = np.array([1, -1, 1])/np.sqrt(3)
a3 = np.array([-1, 1, 1])/np.sqrt(3)
a4 = np.array([-1, -1, 1])/np.sqrt(3)

om_w0 = [0.5, -0.5, 0.5, -0.5] # Initial wheel angular velocities [rad/s]

# Control parameters
nat_freq = 0.05
csi = 1

# Control gain matrices
c1 = 2*nat_freq**2
c2 = csi/nat_freq

A = c1*np.eye(3)
B = c2*np.eye(3)
